In [1]:
from colleges.temple_session import TUSession
from dotenv import load_dotenv
from os import getenv

load_dotenv(override=True)

session = TUSession()
session.login(username=getenv("USERNAME"), password=getenv("PASSWORD"), disable_gui=True)

In [ ]:
session.print_terms(max=2)

In [3]:
from school.course_builder import CourseSelect, CourseIgnore  # noqa: F401

TERM = 202503

SELECTED_COURSES = [
    CourseSelect(course="PHYS1021"),
    CourseSelect(course="MATH2043"),
    CourseSelect(course="IH0851", section="707"),
    CourseSelect(course="CIS1068"),
]

IGNORED_COURSES = [
    CourseIgnore(course="PHYS1021", waitlist=True),
    CourseIgnore(course="MATH2043", waitlist=True),
    CourseIgnore(course="CIS1068", waitlist=True),
]

In [ ]:
from school.course_builder import CourseBuilder
from school.schedule import ScheduleCompare, SchedulePlot

def s_sort(_schedule: SchedulePlot):
    return (
        5 - ScheduleCompare.teacher_rating(_schedule),
        ScheduleCompare.between_total(_schedule),
        # ScheduleCompare.week_range(_schedule),
    )

course_builder = CourseBuilder(session)
course_builder.select_term(TERM)
course_builder.select(SELECTED_COURSES)
course_builder.ignore(IGNORED_COURSES)
course_builder.plot(sort=s_sort, max=10, font_size=10)